<a href="https://colab.research.google.com/github/kangs02/nodes-wheather-app-website/blob/main/Copy_of_Obj_detection_aerial_imagery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#GPU count and name
!nvidia-smi -L

In [ ]:
!pip install Keras==2.7.0rc2

In [ ]:
!python -c 'import keras; print(keras.__version__)'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip /content/drive/MyDrive/RetinanetTest/data.zip -d /content

In [ ]:
import os

repo_url = 'https://github.com/fizyr/keras-retinanet'
repo_dir_path = os.path.abspath(os.path.join('.', os.path.basename(repo_url))) 

!git clone {repo_url} # clone repo

In [ ]:
# from /content/keras-retinanet
%cd {repo_dir_path} 
!pip install .
!python setup.py build_ext --inplace

%cd ..

In [ ]:
ije_repo_url = 'https://github.com/ijeism/ije_retinanet'
ije_repo_dir_path = os.path.abspath(os.path.join('.', os.path.basename(ije_repo_url))) 

!git clone {ije_repo_url} # clone repo
# %cd {ije_repo_dir_path} 
# !git pull # pull repo

In [ ]:
import os
base_dir = os.getcwd()
#base_dir='/content'
base_dir

In [ ]:
# select subsample of N for initial training

from shutil import copyfile
import random

# determine number for subsample
NUM = 80
#NUM=100
# create dir for subsample
sub_dir = base_dir + '/drive/MyDrive/RetinanetTest/images_subset/'
!mkdir {sub_dir}

image_dir = base_dir + '/data/training_data/training_data/images/'
image_paths = os.listdir(image_dir)
random_NUM = random.sample(image_paths, NUM) # randomly select subsample 

# copy subsample into subsample dir
for i in random_NUM:
    copyfile(image_dir + i, sub_dir + i )

In [ ]:
%cd /content/drive/MyDrive/RetinanetTest/images_subset/
!ls -F |grep -v / | wc -l

In [ ]:
!python /content/ije_retinanet/build_dataset.py \
-l /content/data/training_data/training_data/labels/ \
-i /content/drive/MyDrive/RetinanetTest/images_subset/ \
-r /content/drive/MyDrive/RetinanetTest/images_subset/train.csv \
-e /content/drive/MyDrive/RetinanetTest/images_subset/test.csv \
-c /content/drive/MyDrive/RetinanetTest/images_subset/classes.csv 

In [ ]:
import urllib.request

PRETRAINED_MODEL = '/content/keras-retinanet/snapshots/resnet50_coco_best_v2.1.0.h5'

URL_MODEL = 'https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5'

urllib.request.urlretrieve(URL_MODEL, PRETRAINED_MODEL)

In [ ]:
!wc -l /content/drive/MyDrive/RetinanetTest/images_subset/train.csv

In [ ]:
# compute number of annotations to compute number of steps per epoch

import pandas as pd
import math

df = pd.read_csv('/content/drive/MyDrive/RetinanetTest/images_subset/train.csv', header=None)
count = len(df)
#no_steps = math.ceil(count//32)

#hard coded because formula is failing --- FIND SOLUTION
no_steps=20

print("Count of annotations: {}".format(count))
print("Number of steps per epoch: {}".format(no_steps))

In [ ]:
!mkdir /content/run_1/
%cd /content/run_1/

!python /content/keras-retinanet/keras_retinanet/bin/train.py \
--freeze-backbone \
--random-transform \
--weights {PRETRAINED_MODEL} \
--weighted-average \
--batch-size 20 \
--steps {no_steps} \
--epochs 5 \
csv '/content/drive/MyDrive/RetinanetTest/images_subset/train.csv' '/content/drive/MyDrive/RetinanetTest/images_subset/classes.csv'

In [ ]:
# path to trained weights
model_1_path = os.path.join('/content/run_1/snapshots/', sorted(os.listdir('/content/run_1/snapshots/'), reverse=True)[0])
print(model_1_path)

# evaluate model on test.csv
!python /content/keras-retinanet/keras_retinanet/bin/evaluate.py \
csv '/content/drive/MyDrive/RetinanetTest/images_subset/test.csv' \
'/content/drive/MyDrive/RetinanetTest/images_subset/classes.csv' \
{model_1_path} --convert-model

%cd ..

In [ ]:
#run it if pretrained model in present
model_name='resnet50_csv_40.h5'
model_1_path = '/content/drive/MyDrive/RetinanetTest/model/'+model_name
print(model_1_path)

# evaluate model on test.csv
!python /content/keras-retinanet/keras_retinanet/bin/evaluate.py \
csv '/content/drive/MyDrive/RetinanetTest/images_subset/test.csv' \
'/content/drive/MyDrive/RetinanetTest/images_subset/classes.csv' \
{model_1_path} --convert-model

%cd ..

In [ ]:
# create directory with selected test images 

!mkdir /content/data/test_data_images/test_data_images/test
!mv -t /content/data/test_data_images/test_data_images/test \
/content/data/test_data_images/test_data_images/images/0000006_00159_d_0000001.jpg \
/content/data/test_data_images/test_data_images/images/0000006_06773_d_0000018.jpg \
/content/data/test_data_images/test_data_images/images/0000011_00234_d_0000001.jpg \
/content/data/test_data_images/test_data_images/images/0000063_07000_d_0000008.jpg \
/content/data/test_data_images/test_data_images/images/0000127_01002_d_0000123.jpg \
/content/data/test_data_images/test_data_images/images/0000321_00000_d_0000015.jpg

In [ ]:
# print trained model path
model_1_path

In [ ]:
os.getcwd()

In [ ]:
# convert trained model to inference model to be used for generating predictions
!python /content/keras-retinanet/keras_retinanet/bin/convert_model.py '/content/run_1/snapshots/resnet50_csv_05.h5' '/content/run_1/snapshots/resnet50_csv_05_inference.h5'
inference_model = '/content/run_1/snapshots/resnet50_csv_05_inference.h5'

In [ ]:
# convert trained model to inference model to be used for generating predictions [for pretrained model]
!python /content/keras-retinanet/keras_retinanet/bin/convert_model.py '/content/drive/MyDrive/RetinanetTest/model/resnet50_csv_40.h5' '/content/drive/MyDrive/RetinanetTest/model/resnet50_csv_40_inference.h5'
inference_model = '/content/drive/MyDrive/RetinanetTest/model/resnet50_csv_40_inference.h5'

In [ ]:
#run to custom select inference model
inference_model='/content/drive/MyDrive/RetinanetTest/model/resnet50_csv_40_inference.h5'

In [ ]:
# generate prediction files

!python /content/ije_retinanet/image_inference_write.py \
-i '/content/data/test_data_images/test_data_images/test/' \
-t 0.3 \
-m {inference_model} \
-o  /content/data/ \
-l /content/drive/MyDrive/RetinanetTest/images_subset/classes.csv

In [ ]:
os.getcwd()

In [ ]:
# create output directory where you want to save images with bounding boxes

#run 1st

!mkdir /content/data/output

# generate detections on images
!python /content/ije_retinanet/image_inference_print.py \
-i /content/data/test_data_images/test_data_images/test/ \
-t 0.3 \
-m  {inference_model} \
-o /content/data/output

In [ ]:
# combine all csv files into one

#run 2nd

from glob import glob
fout=open("/content/data/output/out.csv","a")
fout.write("path,confidence,ymin,xmin,ymax,xmax,class\n") # create header
for fi in glob("/content/data/*.csv"):
  f = open(fi)
  for line in f:
    fout.write(line)
fout.close()

In [ ]:
from google.colab import files
from glob import glob
# download output images
for p in glob('/content/data/output/*.jpg'):
  files.download(p)
  
# download raw images
for p in glob('/content/data/test_data_images/test_data_images/test/*.jpg'):
  files.download(p)

In [ ]:
# create a dataframe for analysis purposes

import pandas as pd
df = pd.read_csv("/content/data/output/out.csv")
df.head()

In [ ]:
df["class"].value_counts()

In [ ]:
#prediction on videos
#!python drive/MyDrive/RetinanetTest/predict_vids/images_from_vids.py
import cv2
import os

url='drive/MyDrive/RetinanetTest/predict_vids/test_vid2.mp4'
!mkdir /content/video_data
#url='https://drive.google.com/file/d/1fAi1mOf2xxEkZrnWJsFIAdC1l0pwp_M3/view?usp=sharing'
# Opens the Video file
cap= cv2.VideoCapture(url)
if not cap.isOpened():
  print('Failed to open video')
i=0
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == False:
        break
    if i%1 == 0:
      cv2.imwrite('/content/video_data/test_img'+str(i)+'.jpg',frame)
    #cv2.imshow('Image',frame)
    i+=1
    key=cv2.waitKey(50)
    if i==400:
        break
cap.release()
cv2.destroyAllWindows()



In [ ]:
# generate prediction files

!python /content/ije_retinanet/image_inference_write.py \
-i '/content/video_data/' \
-t 0.3 \
-m {inference_model} \
-o  /content/video_data/ \
-l /content/drive/MyDrive/RetinanetTest/images_subset/classes.csv

In [ ]:
# create output directory where you want to save images with bounding boxes

# generate detections on images
!mkdir /content/video_data/output

!python /content/ije_retinanet/image_inference_print.py \
-i /content/video_data/ \
-t 0.3 \
-m  {inference_model} \
-o /content/video_data/output/

In [ ]:
#!python drive/MyDrive/RetinanetTest/predict_vids/output/vid_from_images.py
import cv2
import numpy as np
import glob
 
img_array = []
url='/content/video_data/output'
for filename in sorted(glob.glob(url+'/*.jpg')):
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)

!mkdir /content/video_data/predicted

out = cv2.VideoWriter('/content/video_data/predicted/predicted.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()
